In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from scipy.sparse import coo_matrix

import wals
import model

/home/ville/uni/SHProject/venv/lib64/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ville/uni/SHProject/venv/lib64/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ville/uni/SHProject/venv/lib64/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ville/uni/SHProject/venv/lib64/python3.7/site-

In [2]:
user_map, item_map, train_sparse, test_sparse = model.clean_data("../data/ratings.csv")

In [3]:
latent_factors = 14
num_iters = 20

output_row, output_col = model.train_model(train_sparse, latent_factors, num_iters)

train_rmse = wals.get_rmse(output_row, output_col, train_sparse)
test_rmse = wals.get_rmse(output_row, output_col, test_sparse)
print('Train: ' + str(train_rmse) + ', Test: ' + str(test_rmse))


Train: 1.3742425311004718, Test: 1.936009587635589


In [5]:
user = 0
user_rated = [item_map[i] for i, x in enumerate(user_map) if x == user]
print(user_rated)
model.generate_recommendations(user, user_rated, output_row, output_col, 6)

[0, 2, 5, 43, 46, 62, 89, 97, 124, 130, 136, 184, 190, 197, 201, 224, 257, 275, 291, 307, 314, 320, 325, 367, 384, 398, 418, 436, 461, 476, 484, 485, 508, 509, 510, 513, 520, 546, 551, 559, 592, 594, 615, 632, 701, 705, 720, 723, 734, 767, 781, 782, 783, 786, 787, 788, 789, 797, 801, 810, 815, 819, 827, 828, 830, 835, 855, 862, 897, 898, 899, 906, 908, 910, 913, 914, 919, 920, 922, 924, 926, 938, 954, 956, 963, 968, 973, 976, 980, 989, 995, 1035, 1059, 1075, 1083, 1109, 1125, 1145, 1153, 1170, 1180, 1182, 1189, 1217, 1219, 1223, 1234, 1260, 1297, 1318, 1325, 1331, 1332, 1400, 1406, 1430, 1443, 1474, 1479, 1486, 1492, 1502, 1504, 1515, 1516, 1521, 1525, 1542, 1552, 1555, 1556, 1558, 1561, 1566, 1575, 1576, 1594, 1596, 1598, 1600, 1616, 1627, 1643, 1673, 1686, 1690, 1703, 1733, 1742, 1754, 1767, 1787, 1789, 1795, 1805, 1813, 1825, 1841, 1849, 1857, 1865, 1873, 1882, 1903, 1904, 1916, 1938, 1945, 1956, 1970, 1971, 1978, 1985, 1986, 1989, 1990, 1993, 1996, 2019, 2027, 2037, 2076, 2102, 212

[1066, 659, 901, 507, 915, 2096]